In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import warnings
warnings.filterwarnings('ignore')

from revolve.operators import (
    Operations,
    random_mutation,
    uniform_crossover,
    roulette_wheel_selection,
)

In [2]:
train, test = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)
x_train_full, y_train_full = train
x_test, y_test = test

In [3]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, test_size=0.1, random_state=42)

In [4]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)
x_test = scaler.transform(x_test)

In [5]:
x_test.shape

(102, 13)

In [6]:
y_test.shape

(102,)

In [7]:
y_valid.shape

(41,)

In [8]:
from revolve.grids import MLPParameterGrid
from revolve.architectures import MLPStrategy
from revolve.algorithms import EvolutionaryAlgorithmElitism

In [9]:
operations = Operations()
operations.register(roulette_wheel_selection)
operations.register(uniform_crossover, probability=0.5)
operations.register(random_mutation, probability=0.2)

In [10]:
operations.get_operations()

['selection', 'crossover', 'mutation']

In [11]:
import numpy as np
params = MLPParameterGrid(
    input_shape=(x_train.shape[1],),
    regression_target=1,
    regression_activation= "relu",
    hidden_neurons = [0, 32, 64, 128, 256, 512, 1024, 2048],
    activation='relu',
    dropout=[0.1, 0.2, 0.3, 0.4, 0.5],
    batch_size=[32, 64, 128],
    optimizer='adam',
    learning_rate=[1e-3, 2.5e-4, 5e-4, 7.5e-4, 1e-5],
    l1=1e-5,
    l2=1e-5,
)

In [12]:
params.static_parameters

{'input_shape': (13,),
 'regression_target': 1,
 'regression_activation': 'relu',
 'activation': 'relu',
 'l1': 1e-05,
 'l2': 1e-05,
 'optimizer': 'adam'}

In [13]:
params.learnable_parameters

{'hidden_neurons': [0, 32, 64, 128, 256, 512, 1024, 2048],
 'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
 'batch_size': [32, 64, 128],
 'learning_rate': [0.001, 0.00025, 0.0005, 0.00075, 1e-05]}

In [14]:
callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=2)

mlp = MLPStrategy(
    max_fc = 3,
    squeeze_fc=False,
    epochs=50,
    callback=callback,
    loss='mean_squared_error',
    metric = tfa.metrics.RSquare(),
    parameters = params
)

In [15]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
valid_data = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))

data = (train_data, valid_data, test_data)

In [16]:
ga = EvolutionaryAlgorithmElitism(
    strategy=mlp,
    pop_size=30,
    elitism_size=2,
    operations=operations,
)

In [17]:
best_chromosome = ga.fit(
    data,
    generations=3,
)

  0%|          | 0/30 [00:00<?, ?it/s]

Generation 0,                 Best error: 22.87187385559082,                 Best R2 0.7263087034225464


  0%|          | 0/30 [00:00<?, ?it/s]

Generation 1,                 Best error: 21.207847595214844,                 Best R2 0.7476918697357178


  0%|          | 0/30 [00:00<?, ?it/s]

Generation 2,                 Best error: 20.2978572845459,                 Best R2 0.7568013668060303


In [18]:
model = best_chromosome.decode(params)

In [19]:
df = ga.results_df()

In [20]:
df.head()

,learnt_parameters,static_parameters,loss,metric,generation
0,"{'fc_0': {'hidden_neurons': 64, 'activation': ...","{'input_shape': (13,), 'regression_target': 1,...",23.340296,0.725512,0
1,"{'fc_0': {'hidden_neurons': 2048, 'activation'...","{'input_shape': (13,), 'regression_target': 1,...",25.635225,0.693927,0
2,"{'fc_0': {'hidden_neurons': 512, 'activation':...","{'input_shape': (13,), 'regression_target': 1,...",23.717901,0.716334,0
3,"{'fc_0': {'hidden_neurons': 2048, 'activation'...","{'input_shape': (13,), 'regression_target': 1,...",29.516582,0.645864,0
4,"{'fc_0': {'hidden_neurons': 64, 'activation': ...","{'input_shape': (13,), 'regression_target': 1,...",25.163296,0.700056,0


In [21]:
best_chromosome_row = df[df.loss == df.loss.min()]

In [31]:
dict(best_chromosome_row.learnt_parameters)

{71: {'fc_0': {'hidden_neurons': 128,
   'activation': 'relu',
   'dropout': 0.3,
   'l1': 1e-05,
   'l2': 1e-05},
  'fc_1': {'hidden_neurons': 128,
   'activation': 'relu',
   'dropout': 0.5,
   'l1': 1e-05,
   'l2': 1e-05},
  'fc_2': {'hidden_neurons': 64,
   'activation': 'relu',
   'dropout': 0.2,
   'l1': 1e-05,
   'l2': 1e-05},
  'batch_size_3': {'batch_size': 64},
  'optimizer_4': {'optimizer': 'adam'},
  'learning_rate_5': {'learning_rate': 1e-05}}}

In [33]:
elite_models = ga.elite_models

In [34]:
elite_models